In [2]:
"""
Lab 6 - COEN 140
Marianne Fuyu Yamazaki Dorr
10/25/2022

Implement Cross Validation and Gradient Descent in Python.  
"""

import numpy as np
import pandas as pd
import math

def get_data(file):
    return pd.read_csv(file ,delimiter='\t')

def RMSE(actual, predicted):
    m = len(predicted)
    dif = (actual - predicted)
    sumsquared = 0
    
    for x in dif:
        sumsquared += x**2
    
    rmse = math.sqrt(sumsquared/m)
    return rmse

train = get_data("crime-train.txt")
test = get_data ("crime-test.txt")

#Training Data
#add training data except for output to x
x_train = train.drop('ViolentCrimesPerPop', axis = 1)
#add dummy column of 1s to solve for intercept b
dummy = np.ones(len(x_train))
x_train = np.column_stack((x_train, dummy))
#ensure that data is of correct type float64
x_train = np.float64(x_train)

#add output data to y, first column of train
y_train = train['ViolentCrimesPerPop']
#ensure that data is of correct type float64
y_train = np.float64(y_train)
y_train = np.reshape(y_train, (-1,1))
#Testing Data
#add testing data except for output to x
x_test = test.drop('ViolentCrimesPerPop', axis = 1)
dummy2 = np.ones(len(x_test))
x_test = np.column_stack((x_test, dummy2))
x_test = np.float64(x_test)

y_test = test['ViolentCrimesPerPop']
y_test = np.float64(y_test)
y_test = np.reshape(y_test, (-1,1))

"""
PART I - Ridge Regression 
with k-folding = 5
"""
print("PART I - Ridge Regression")
print()
def ridge_reg(x, y, val, l):
    #identity matrix must be of size (p+1) by (p+1)
    #(xTx + lambdaI)-1xTy
    #use np.identity(size) to create identity matrix
    lambda_matrix = l * np.identity(len(x_train.T))
    first = np.linalg.inv(np.dot(x.T, x) + lambda_matrix)
    second = np.dot(x.T, y)
    w = np.dot(first, second)
    
    pred = []
    
    for v in val:
        pred.append(np.dot(v.T, w))
    
    return pred

l = 400
lval = [0]*10
errors = np.zeros((10, 5))

for x in range(10):
    for k in range(5):
        x_traink = np.concatenate((x_train[0:int(k*(len(x_train)/5))], x_train[int((k+1)*(len(x_train)/5)):int(len(x_train))]))
        y_traink = np.concatenate((y_train[0:int(k*(len(y_train)/5))], y_train[int((k+1)*(len(y_train)/5)):int(len(y_train))]))
        
        x_valk = x_train[int(k*(len(x_train)/5)):int((k+1)*(len(x_train)/5))]
        y_valk = y_train[int(k*(len(y_train)/5)):int((k+1)*(len(y_train)/5))]
        
        pred = ridge_reg(x_traink, y_traink, x_valk, l)

        errors[x, k] = RMSE(pred, y_valk)
    lval[x] = l   
    l = l/2

avg = [0]*10
for x in range(10):
    sum = 0
    for y in range(5):
        sum += errors[x][y]
    avg[x] = (sum/5)
    print("Value of lambda: {} and average RMSE: {}".format(lval[x], avg[x]))

minval = min(avg)
minposition = avg.index(minval)
best_l =  lval[minposition]
print()
print("Choose {} for lambda since it provides the lowest avg error.".format(lval[minposition]))
print()

"""
PART II - Linear Regression with Gradient Descent

"""
print("PART II - Linear Regression with Gradient Descent")

#random w as the initial condition
def lin_reg_gd(x, y, w):
    w_next = w + (alpha*np.dot(x.T, (y-np.dot(x, w))))
    return w_next

def lin_reg_loss(x, y, w):
    loss = np.dot((y-np.dot(x, w)).T, (y-np.dot(x,w)))
    return loss

alpha = 0.00001
L1 = 1
L0 = 0

w0 = np.random.normal(0, 1, (x_train.shape[1], 1))

while (abs(L1-L0) > (0.00001)):
    w1 = lin_reg_gd(x_train, y_train, w0)
    L0 = lin_reg_loss(x_train, y_train, w0)
    L1 = lin_reg_loss(x_train, y_train, w1)
    w0 = w1

def problem2(w, m):
    #y = xTw
    pred = []
    for x in m:
        pred.append(np.dot(w.T, x))
    return pred

#find predicted y of training data
train_pred = []
test_pred = []
train_pred = problem2(w0, x_train)
print()
train_error = RMSE(train_pred, y_train)
print("RMSE of Linear Regression Model on Training Data: {}".format(train_error))
print()

#find predicted y of testing data

test_pred = problem2(w0, x_test)
test_error = RMSE(test_pred, y_test)
print("RMSE of Linear Regression Model on Testing Data: {}".format(test_error))
print()

print("RMSEs of Lab 5: ")
print()
print("RMSE of Linear Regression Model on Training Data: 0.12768967421762187")
print()
print("RMSE of Linear Regression Model on Testing Data: 0.14583464490949136")
print()

"""
PART III - Ridge regression with 5-fold cross valuation using the gradient descent algorithm
"""
print("Part III - Ridge regression with 5-fold cross valuation using the gradient descent algorithm")
print()
def ridge_reg_gd(x, y, w, l):
    first = (y - np.dot(x, w)).T
    second = (y-np.dot(x, w))
    new_w = np.dot(first, second) + (best_l * np.dot(w0.T, w0))
    
    return new_w

alpha = 0.00001

def problem3(w, val):
    pred = []
    for x in val:
        pred.append(np.dot(w.T, x))
    return pred

l = 400
lval = [0]*10
errors = np.zeros((10, 5))

for x in range(10):
    for k in range(5):
        x_traink = np.concatenate((x_train[0:int(k*(len(x_train)/5))], x_train[int((k+1)*(len(x_train)/5)):int(len(x_train))]))
        y_traink = np.concatenate((y_train[0:int(k*(len(y_train)/5))], y_train[int((k+1)*(len(y_train)/5)):int(len(y_train))]))
        
        x_valk = x_train[int(k*(len(x_train)/5)):int((k+1)*(len(x_train)/5))]
        y_valk = y_train[int(k*(len(y_train)/5)):int((k+1)*(len(y_train)/5))]
        
        L1 = 1
        L0 = 0
        w0 = np.random.normal(0, 1, (x_train.shape[1],1))
        
        while (abs(L1-L0) > (0.00001)):
            w1 = w0 - (alpha*(np.dot(x_traink.T, (np.dot(x_traink, w0) - y_traink)) + (l * w0)))
    
            L0 = ridge_reg_gd(x_traink, y_traink, w0, l)
            L1 = ridge_reg_gd(x_traink, y_traink, w1, l)
            w0 = w1
        
        val_results = []
        val_results = problem3(w0, x_valk)
        
        errors[x][k] = RMSE(val_results, y_valk)
    lval[x] = l   
    l = l/2

new_avg = [0]*10
for x in range(10):
    sum = 0
    for y in range(5):
        sum += errors[x][y]
    new_avg[x] = (sum/5)
    print("Value of lambda: {} and average RMSE: {}".format(lval[x], new_avg[x]))

minval = min(new_avg)
minposition = new_avg.index(minval)
best_l =  lval[minposition]
print()
print("Choose {} for lambda since it has the lowest average error for all error rates".format(best_l))
print()
#for testing data
L1 = 1
L0 = 0
# generate a Gaussian (normal) distribution for our initial value, based off how many features we have
w0 = np.random.normal(0, 1, (x_train.shape[1],1))
while (abs(L1 - L0) > 0.00001):
    w1 = w0 - (alpha * (np.dot(x_train.T, (np.dot(x_train, w0) - y_train)) + (best_l * w0)))
    L0 = ridge_reg_gd(x_train, y_train, w0, best_l)
    L1 = ridge_reg_gd(x_train, y_train, w1, best_l)
    w0 = w1

test_pred = []
test_pred = problem3(w0, x_test)

test_ridge_error = RMSE(test_pred, y_test)
print("RMSE for test data using ridge regression with gradient descent is ",test_ridge_error)

PART I - Ridge Regression

Value of lambda: 400 and average RMSE: 0.14951277619075795
Value of lambda: 200.0 and average RMSE: 0.14069401845317783
Value of lambda: 100.0 and average RMSE: 0.13727677223186574
Value of lambda: 50.0 and average RMSE: 0.1361559439275552
Value of lambda: 25.0 and average RMSE: 0.13591585919851984
Value of lambda: 12.5 and average RMSE: 0.13602237446230686
Value of lambda: 6.25 and average RMSE: 0.13626786494076748
Value of lambda: 3.125 and average RMSE: 0.13657095693642846
Value of lambda: 1.5625 and average RMSE: 0.1368876407319742
Value of lambda: 0.78125 and average RMSE: 0.13717626806866692

Choose 25.0 for lambda since it provides the lowest avg error.

PART II - Linear Regression with Gradient Descent

RMSE of Linear Regression Model on Training Data: 0.1285257257429593

RMSE of Linear Regression Model on Testing Data: 0.14864482169178425

RMSEs of Lab 5: 

RMSE of Linear Regression Model on Training Data: 0.12768967421762187

RMSE of Linear Regressi